Load environment variables from a file `.env` in the current directory.
You can take a look at the `.env.template` file to see how it should look like.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv("AZURE_OPENAI_ENDPOINT"))
print(os.getenv("AZURE_OPENAI_API_KEY"))
print(os.getenv("OPENAI_API_VERSION"))

https://crgar-openai-openai.openai.azure.com/
None
2023-10-01-preview


In [2]:
# Create a prompt template
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="podcast player"))

What is a good name for a company that makes podcast player?


In [3]:
# Call the LLM and run the prompt
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain

llm = AzureChatOpenAI(
    #azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    #api_version=os.getenv("OPENAI_API_VERSION"),
    #model_name=os.getenv("OPENAI_MODEL_NAME"),
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
    temperature=0.9) #temperature dictates how whacky the output should be
llmchain = LLMChain(llm=llm, prompt=prompt)
llmchain.run("podcast player")

'PodPlay Inc.'

In [4]:
# Trying a stock chain: PALChain. This requires langchain-experimental package to be installed
from langchain_experimental.pal_chain import PALChain
palchain = PALChain.from_math_prompt(llm=llm, verbose=True)
palchain.run("If my age is half of my dad's age and he is going to be 60 next year, what is my current age?")



> Entering new PALChain chain...


Python REPL can execute arbitrary code. Use with caution.


def solution():
    """If my age is half of my dad's age and he is going to be 60 next year, what is my current age?"""
    dad_age_next_year = 60
    my_age = dad_age_next_year // 2
    result = my_age
    return result

> Finished chain.


'30'

In [14]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import AzureChatOpenAI
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
    temperature=0)

tools = [YahooFinanceNewsTool()]

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

agent.run(input="Can you show gain or loss percentage of Microsoft stock?", verbose=True)



> Entering new AgentExecutor chain...
I need to find financial news about Microsoft stocks to answer this question.
Action: yahoo_finance_news
Action Input: MSFT
Observation: One of This Year's Top-Performing Dow Stocks Could Continue Its Hot Streak in 2024
The Dow Jones Industrial Average is underperforming both the Nasdaq Composite and the S&P 500 in 2023.  At the time of this writing, the top-four best-performing Dow stocks so far this year are all tech stocks.  Salesforce is up 89% year to date (YTD), Intel is up roughly 59%, Microsoft (NASDAQ: MSFT) is up 55%, and Apple is up roughly 49%.

Is Microsoft (MSFT) Still a Great Investment Choice?
White Brook Capital Partners, an investment management firm, released its third quarter 2023 investor letter, a copy of which can be downloaded here. White Brook Capital Partners maintains its outperformance against the S&P Midcap 400 year to date, boasting a 6.49% increase compared to the index’s 4.28% gain. However, it falls short of the S

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I should try searching for a different news source or website to find the information I need.
Action: search for Microsoft stock news on a different financial news website`